In [5]:
import pandas as pd
import numpy as np

In [2]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [4]:
from sklearn.model_selection import train_test_split

# X - матрица признаков, y - целевая переменная
X = data.drop('MEDV', axis=1)
y = data['MEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RANDOM_STATE)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((404, 13), (102, 13), (404,), (102,))

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

lr = LinearRegression()
lr.fit(X_train, y_train)


r2_lr = lr.score(X_test, y_test)

ridge = Ridge()
ridge.fit(X_train, y_train)
r2_ridge = ridge.score(X_test, y_test)

lasso = Lasso()
lasso.fit(X_train, y_train)
r2_lasso = lasso.score(X_test, y_test)

print(f"Linear regression R2 score: {r2_lr}")
print(f"Ridge regression R2 score: {r2_ridge}")
print(f"Lasso regression R2 score: {r2_lasso}")


results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]

Linear regression R2 score: 0.668482575397159
Ridge regression R2 score: 0.6659608075261695
Lasso regression R2 score: 0.6668687223368214


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [49]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import RidgeCV, LassoCV

# Создаем объект модели
ridge = Ridge()
lasso = Lasso()

# Создаем сетку параметров для перебора
param_grid = {'alpha': [10**i for i in range(-5, 6)]}

# Инициализируем объект GridSearchCV
ridge_grid_search_cv = GridSearchCV(ridge, param_grid, cv=5)
lasso_grid_search_cv = GridSearchCV(lasso, param_grid, cv=5)

# Обучаем модели на обучающей выборке
ridge_grid_search_cv.fit(X_train, y_train)
lasso_grid_search_cv.fit(X_train, y_train)

r2_ridge_grid_search = ridge_grid_search_cv.score(X_test, y_test)
r2_lasso_grid_search = lasso_grid_search_cv.score(X_test, y_test)

# Выводим наилучшие параметры и R2 на тестовой выборке
print("Ridge GridSearchCV : Best alpha =", ridge_grid_search_cv.best_params_['alpha'])
print("Lasso GridSearchCV: Best alpha =", lasso_grid_search_cv.best_params_['alpha'])


# Инициализируем объекты RidgeCV и LassoCV
ridge_cv = RidgeCV(alphas=[10**i for i in range(-5, 6)], cv=5)
lasso_cv = LassoCV(alphas=[10**i for i in range(-5, 6)], cv=5)

# Обучаем модели на обучающей выборке
ridge_cv.fit(X_train, y_train)
lasso_cv.fit(X_train, y_train)

r2_ridge_cv = ridge_cv.score(X_test, y_test)
r2_lasso_cv = lasso_cv.score(X_test, y_test)


# Выводим наилучшие параметры и R2 на тестовой выборке
print("Ridge RidgeCV: Best alpha =", ridge_cv.alpha_)
print("Lasso LassoCV: Best alpha =", lasso_cv.alpha_)


print(f"r2_ridge: {r2_ridge}")
print(f"r2_ridge_grid_search: {r2_ridge_grid_search}")
print(f"r2_ridge_cv: {r2_ridge_cv}")

print("------")

print(f"r2_lasso: {r2_lasso}")
print(f"r2_lasso_grid_search: {r2_lasso_grid_search}")
print(f"r2_lasso_cv: {r2_lasso_cv}")

results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]

Ridge GridSearchCV : Best alpha = 1e-05
Lasso GridSearchCV: Best alpha = 1e-05
Ridge RidgeCV: Best alpha = 1e-05
Lasso LassoCV: Best alpha = 1e-05
r2_ridge: 0.6659608075261695
r2_ridge_grid_search: 0.6684825680074258
r2_ridge_cv: 0.6684825680074258
------
r2_lasso: 0.6668687223368214
r2_lasso_grid_search: 0.668482959588568
r2_lasso_cv: 0.668482959588568


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [57]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score

# Создаем Pipeline для масштабирования данных и обучения Ridge/Lasso модели
pipeline_ridge_standard_scaler = Pipeline([('scaler', StandardScaler()), ('reg', Ridge())])
pipeline_lasso_standard_scaler = Pipeline([('scaler', StandardScaler()), ('reg', Lasso())])

# Обучаем модели
pipeline_ridge_standard_scaler.fit(X_train, y_train)
pipeline_lasso_standard_scaler.fit(X_train, y_train)

# Вычисляем R2 на тестовой выборке
y_predict_ridge_standard_scaler = pipeline_ridge_standard_scaler.predict(X_test)
y_predict_lasso_standard_scaler = pipeline_lasso_standard_scaler.predict(X_test)

r2_ridge_standart_scaler = r2_score(y_test, y_predict_ridge_standard_scaler)
r2_lasso_standart_scaler = r2_score(y_test, y_predict_lasso_standard_scaler)


# Создаем Pipeline с MinMaxScaler и Ridge/Lasso
pipeline_ridge_min_max_scaler = Pipeline([('scaler', MinMaxScaler()), ('reg', Ridge())])
pipeline_lasso_min_max_scaler = Pipeline([('scaler', MinMaxScaler()), ('reg', Lasso())])


# Обучаем модели
pipeline_ridge_min_max_scaler.fit(X_train, y_train)
pipeline_lasso_min_max_scaler.fit(X_train, y_train)

# Вычисляем R2 на тестовой выборке
y_predict_ridge_min_max_scaler = pipeline_ridge_min_max_scaler.predict(X_test)
y_predict_lasso_min_max_scaler = pipeline_lasso_min_max_scaler.predict(X_test)
r2_ridge_min_max_scaler = r2_score(y_test, y_predict_ridge_min_max_scaler)
r2_lasso_min_max_scaler = r2_score(y_test, y_predict_lasso_min_max_scaler)


print(f"r2_ridge: {r2_ridge}")
print(f"r2_ridge_grid_search: {r2_ridge_grid_search}")
print(f"r2_ridge_cv: {r2_ridge_cv}")
print(f"r2_ridge_standart_scaler: {r2_ridge_standart_scaler}")
print(f"r2_ridge_min_max_scaler: {r2_ridge_min_max_scaler}")

print("------")

print(f"r2_lasso: {r2_lasso}")
print(f"r2_lasso_grid_search: {r2_lasso_grid_search}")
print(f"r2_lasso_cv: {r2_lasso_cv}")
print(f"r2_lasso_standart_scaler: {r2_lasso_standart_scaler}")
print(f"r2_lasso_min_max_scaler: {r2_lasso_min_max_scaler}")


results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]

r2_ridge: 0.6659608075261695
r2_ridge_grid_search: 0.6684825680074258
r2_ridge_cv: 0.6684825680074258
r2_ridge_standart_scaler: 0.668190107677443
r2_ridge_min_max_scaler: 0.6762207658974593
------
r2_lasso: 0.6668687223368214
r2_lasso_grid_search: 0.668482959588568
r2_lasso_cv: 0.668482959588568
r2_lasso_standart_scaler: 0.624044752347846
r2_lasso_min_max_scaler: 0.2573921442545195


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [68]:
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Создаем объекты RidgeCV и LassoCV
ridge_cv = RidgeCV(alphas=[10**i for i in range(-5, 6)], cv=5)
lasso_cv = LassoCV(alphas=[10**i for i in range(-5, 6)], cv=5)

# Масштабируем данные
standard_scaler = StandardScaler()
X_train_standard_scaled = standard_scaler.fit_transform(X_train)
X_test_standard_scaled = standard_scaler.transform(X_test)

# Обучаем модели на масштабированных данных
ridge_cv.fit(X_train_standard_scaled, y_train)
lasso_cv.fit(X_train_standard_scaled, y_train)

# Получаем значения R2 для Ridge и Lasso на тестовой выборке
r2_ridge_standart_scaler_cv = ridge_cv.score(X_test_standard_scaled, y_test)
r2_lasso_standart_scaler_cv = lasso_cv.score(X_test_standard_scaled, y_test)

# Выводим наилучшие параметры и R2 на тестовой выборке
print("Ridge RidgeCV StandardScaler: Best alpha =", ridge_cv.alpha_)
print("Lasso LassoCV StandardScaler: Best alpha =", lasso_cv.alpha_)

# Выводим значения R2
print("r2_ridge_standart_scaler_cv: ", r2_ridge_standart_scaler_cv)
print("r2_lasso_standart_scaler_cv: ", r2_lasso_standart_scaler_cv)

print("------")

# Масштабируем данные
min_max_scaler = MinMaxScaler()
X_train_min_max_scaled = min_max_scaler.fit_transform(X_train)
X_test_min_max_scaled = min_max_scaler.transform(X_test)

# Обучаем модели на масштабированных данных
ridge_cv.fit(X_train_min_max_scaled, y_train)
lasso_cv.fit(X_train_min_max_scaled, y_train)

# Получаем значения R2 для Ridge и Lasso на тестовой выборке
r2_ridge_min_max_scaler_cv = ridge_cv.score(X_test_standard_scaled, y_test)
r2_lasso_min_max_scaler_cv = lasso_cv.score(X_test_standard_scaled, y_test)

# Выводим наилучшие параметры и R2 на тестовой выборке
print("Ridge RidgeCV MinMaxScaler: Best alpha =", ridge_cv.alpha_)
print("Lasso LassoCV MinMaxScaler: Best alpha =", lasso_cv.alpha_)

# Выводим значения R2
print("r2_ridge_min_max_scaler_cv: ", r2_ridge_min_max_scaler_cv)
print("r2_lasso_min_max_scaler_cv: ", r2_lasso_min_max_scaler_cv)


results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]

Ridge RidgeCV StandardScaler: Best alpha = 1.0
Lasso LassoCV StandardScaler: Best alpha = 1e-05
r2_ridge_standart_scaler_cv:  0.6681901076774431
r2_lasso_standart_scaler_cv:  0.6684821312777706
------
Ridge RidgeCV MinMaxScaler: Best alpha = 0.1
Lasso LassoCV MinMaxScaler: Best alpha = 1e-05
r2_ridge_min_max_scaler_cv:  -20.660347184502935
r2_lasso_min_max_scaler_cv:  -21.108987672687494


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [75]:
from sklearn.preprocessing import PolynomialFeatures

# Создаем объект PolynomialFeatures для добавления попарных произведений и квадратов признаков
poly = PolynomialFeatures(include_bias=False)

# Масштабируем данные
standard_scaler = StandardScaler()
X_train_standard_scaled = standard_scaler.fit_transform(X_train)
X_test_standard_scaled = standard_scaler.transform(X_test)

# Добавляем попарные произведения и квадраты признаков
X_train_poly = poly.fit_transform(X_train_standard_scaled)
X_test_poly = poly.transform(X_test_standard_scaled)

# Масштабируем признаки с помощью StandardScaler
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

# Обучаем модель Ridge с параметрами по умолчанию
ridge = Ridge()
ridge.fit(X_train_scaled, y_train)
r2_ridge_standart_scaler_poly = ridge.score(X_test_scaled, y_test)
print(f'r2_ridge_standart_scaler_poly: {r2_ridge_standart_scaler_poly}')

# Обучаем модель Lasso с параметрами по умолчанию
lasso = Lasso()
lasso.fit(X_train_scaled, y_train)
r2_lasso_standart_scaler_poly = lasso.score(X_test_scaled, y_test)
print(f'r2_lasso_standart_scaler_poly: {r2_lasso_standart_scaler_poly}')

print("------")

# Масштабируем данные MinMaxScaler
min_max_scaler = MinMaxScaler()
X_train_min_max_scaled = min_max_scaler.fit_transform(X_train)
X_test_min_max_scaled = min_max_scaler.transform(X_test)

# Добавляем попарные произведения и квадраты признаков
X_train_poly = poly.fit_transform(X_train_min_max_scaled)
X_test_poly = poly.transform(X_test_min_max_scaled)

# Масштабируем признаки с помощью StandardScaler
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

# Обучаем модель Ridge с параметрами по умолчанию
ridge = Ridge()
ridge.fit(X_train_scaled, y_train)
r2_ridge_min_max_scaler_poly = ridge.score(X_test_scaled, y_test)
print(f'r2_ridge_min_max_scaler_poly: {r2_ridge_min_max_scaler_poly}')

# Обучаем модель Lasso с параметрами по умолчанию
lasso = Lasso()
lasso.fit(X_train_scaled, y_train)
r2_lasso_min_max_scaler_poly = lasso.score(X_test_scaled, y_test)
print(f'r2_lasso_min_max_scaler_poly: {r2_lasso_min_max_scaler_poly}')




results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]

r2_ridge_standart_scaler_poly: 0.8172321234956248
r2_lasso_standart_scaler_poly: 0.7388607450771216
------
r2_ridge_min_max_scaler_poly: 0.8309016090915333
r2_lasso_min_max_scaler_poly: 0.7319986508747183


7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [112]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

# Создаем пайплайн для масштабирования, генерации полиномиальных признаков и моделирования
pipe_standard_scaler = make_pipeline(StandardScaler(), PolynomialFeatures())

ridge_cv = RidgeCV(alphas=[10**i for i in range(-5, 6)], cv=5)
lasso_cv = LassoCV(alphas=[10**i for i in range(-5, 6)], cv=5)

# Масштабируем данные
X_train_standard_scaled = pipe_standard_scaler.fit_transform(X_train)
X_test_standard_scaled = pipe_standard_scaler.transform(X_test)

# Обучаем модели на масштабированных данных
ridge_cv.fit(X_train_standard_scaled, y_train)
lasso_cv.fit(X_train_standard_scaled, y_train)

# Получаем значения R2 для Ridge и Lasso на тестовой выборке
r2_ridge_standart_scaler_poly_cv = ridge_cv.score(X_test_standard_scaled, y_test)
r2_lasso_standart_scaler_poly_cv = lasso_cv.score(X_test_standard_scaled, y_test)

# Выводим наилучшие параметры и R2 на тестовой выборке
print("Ridge RidgeCV StandardScaler: Best alpha =", ridge_cv.alpha_)
print("Lasso LassoCV StandardScaler: Best alpha =", lasso_cv.alpha_)

# Выводим значения R2
print(f'r2_ridge_standart_scaler_poly_cv: {r2_ridge_standart_scaler_poly_cv}')
print(f'r2_lasso_standart_scaler_poly_cv: {r2_lasso_standart_scaler_poly_cv}')
print("------")


pipe_min_max_scaler = make_pipeline(MinMaxScaler(), PolynomialFeatures())
X_train_min_max_scaled = pipe_min_max_scaler.fit_transform(X_train)
X_test_min_max_scaled = pipe_min_max_scaler.transform(X_test)
ridge_cv = RidgeCV(alphas=[10**i for i in range(-5, 6)], cv=5)
lasso_cv = LassoCV(alphas=[10**i for i in range(-5, 6)], cv=5)
ridge_cv.fit(X_train_min_max_scaled, y_train)
lasso_cv.fit(X_test_min_max_scaled, y_train)
r2_ridge_min_max_scaler_poly_cv = ridge_cv.score(X_test_min_max_scaled, y_test)
r2_lasso_min_max_scaler_poly_cv = lasso_cv.score(X_test_min_max_scaled, y_test)
print("Ridge RidgeCV MinMaxScaler: Best alpha =", ridge_cv.alpha_)
print("Lasso LassoCV MinMaxScaler: Best alpha =", lasso_cv.alpha_)
print(f'r2_ridge_min_max_scaler_poly_cv: {r2_ridge_min_max_scaler_poly_cv}')
print(f'r2_lasso_min_max_scaler_poly_cv: {r2_lasso_min_max_scaler_poly_cv}')


results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]

D:\soft\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.708888102574747, tolerance: 2.8821090464396293
  model = cd_fast.enet_coordinate_descent_gram(
D:\soft\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 325.3702094752035, tolerance: 2.8821090464396293
  model = cd_fast.enet_coordinate_descent_gram(
D:\soft\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:525: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 649.714807377882, tolerance: 2.8821090464396293
  model = cd_fast.enet_coordinate_descent_gram(
D:\soft\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:525: ConvergenceWarning: Objective di

Ridge RidgeCV StandardScaler: Best alpha = 10.0
Lasso LassoCV StandardScaler: Best alpha = 0.1
r2_ridge_standart_scaler_poly_cv: 0.8187344606117982
r2_lasso_standart_scaler_poly_cv: 0.8128138856150267
------


ValueError: X and y have inconsistent dimensions (102 != 404)

8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [115]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV

# Создаем Pipeline
pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('poly', PolynomialFeatures()), 
    ('reg', Ridge())
])

# Создаем сетку параметров, которые будем перебирать в GridSearchCV
param_grid = {
    'poly__degree': [1, 2, 3],  # Степень полинома
    'scaler': [StandardScaler(), MinMaxScaler()],  # Метод масштабирования
    'reg': [Ridge(), Lasso()],  # Тип регуляризации
    'reg__alpha': [0.1, 1, 10]  # Коэффициент регуляризации
}

# Создаем GridSearchCV с заданными параметрами
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='r2')

# Обучаем модель и выводим лучшие параметры и результат R2
grid.fit(X_train, y_train)

print('Параметры лучшей модели:\n', grid.best_params_)
r2_best_model = grid.best_score_
print('r2_best_model', r2_best_model)
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]

Параметры лучшей модели:
 {'poly__degree': 3, 'reg': Ridge(alpha=0.1), 'reg__alpha': 0.1, 'scaler': MinMaxScaler()}
r2_best_model 0.8662481490843126


In [116]:
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668483
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.66819
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


http://archive.ics.uci.edu/ml/datasets/Adult

In [10]:
data = pd.read_csv('adult.csv')
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [11]:
# Назначение имен столбцов

# Замена целевой переменной на числовые значения
data['class'] = data['class'].map({'<=50K': 1, '>50K': 0})

# Выделение признаков и целевой переменной
X = data.drop('class', axis=1)
y = data['class']

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [13]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# предсказание самого частого класса
y_pred = np.zeros_like(y_test) + y_train.value_counts().idxmax()

# y_test - истинные значения целевой переменной на тестовой выборке
# y_pred - предсказанные значения

acc_most_frequent = accuracy_score(y_test, y_pred)
f1_most_frequent = f1_score(y_test, y_pred)
print(f"acc_most_frequent: {acc_most_frequent}")
print(f"f1_most_frequent: {f1_most_frequent}")

results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]

acc_most_frequent: 0.7581382652016652
f1_most_frequent: 0.8624330409129726


NameError: name 'results_classification' is not defined

11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [124]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')
X_imputed = imputer.fit_transform(X)
print(f"X_imputed: {X_imputed}")

X_imputed: [[39 'State-gov' 77516 ... 0 40 'United-States']
 [50 'Self-emp-not-inc' 83311 ... 0 13 'United-States']
 [38 'Private' 215646 ... 0 40 'United-States']
 ...
 [38 'Private' 374983 ... 0 50 'United-States']
 [44 'Private' 83891 ... 0 40 'United-States']
 [35 'Self-emp-inc' 182148 ... 0 60 'United-States']]


12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [130]:

numerical_columns = data.select_dtypes(include='number').columns

categorical_columns = data.select_dtypes(include='object').columns

# cat_features = data.select_dtypes(exclude='number')

print(f"numerical_columns: {numerical_columns}")
print(f"categorical_columns: {categorical_columns}")
# print(f"cat_features: {cat_features}")

numerical_columns: Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'class'],
      dtype='object')
categorical_columns: Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')


13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler , OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.impute import SimpleImputer


data = pd.read_csv('adult.csv')
data['class'] = data['class'].map({'<=50K': 1, '>50K': 0})
X = data.drop('class', axis=1)
y = data['class']

num_cols = X.select_dtypes(include='number').columns.tolist()
cat_cols = X.select_dtypes(include='object').columns.tolist()
print(num_cols)
print(cat_cols)

# Define preprocessing steps for numerical and categorical features
numerical_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])

categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)])


# Define classifiers
lr = LogisticRegression()
knn = KNeighborsClassifier()
svc = LinearSVC()

# Define pipeline with column transformer and classifiers
pipe_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', MinMaxScaler()),
    ('classifier', lr)
])

pipe_knn = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', MinMaxScaler()),
    ('classifier', knn)
])

pipe_svc = Pipeline([
    ('preprocessor', preprocessor),
    ('scaler', MinMaxScaler()),
    ('classifier', svc)
])

# Calculate cross-validation scores using accuracy and F1 metrics
# acc_LR = cross_val_score(pipe_lr, X, y, cv=5, scoring='accuracy')
# f1_LR = cross_val_score(pipe_lr, X, y, cv=5, scoring='f1')

# acc_KNN = cross_val_score(pipe_knn, X, y, cv=5, scoring='accuracy')
# f1_KNN = cross_val_score(pipe_knn, X, y, cv=5, scoring='f1')

# acc_SVM = cross_val_score(pipe_svc, X, y, cv=5, scoring='accuracy')
# f1_SVM = cross_val_score(pipe_svc, X, y, cv=5, scoring='f1')

# print(f"acc_LR: {acc_LR}")
# print(f"acc_KNN: {acc_KNN}")
# print(f"acc_SVM: {acc_SVM}")

# print(f"f1_LR: {f1_LR}")
# print(f"f1_KNN: {f1_KNN}")
# print(f"f1_SVM: {f1_SVM}")
                                           
                                           
# results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
# results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
# results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]

['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']


D:\soft\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\soft\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\soft\Anaconda\lib\site-packages\sklearn\pipeline.py", line 330, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "D:\soft\Anaconda\lib\site-packages\sklearn\pipeline.py", line 292, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "D:\soft\Anaconda\lib\site-packages\joblib\memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "D:\soft\Anaconda\lib\site-packages\sklearn\pipeline.py", line 740, in _fit_transform_one
    res = transformer.fit_transform(X, y, **fit_params)
  File "D:\soft\Anaconda\lib\site-packages\sklearn\base

In [23]:
y.value_counts()

1    37155
0    11687
Name: class, dtype: int64

In [24]:
X

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States


In [169]:
X.select_dtypes(include=['number']).columns

Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')

In [168]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
import pandas as pd

# Load data and separate target variable
data = pd.read_csv('adult.csv')
data['class'] = data['class'].map({'<=50K': 1, '>50K': 0})
X = data.drop('class', axis=1)
y = data['class']

# Define preprocessing steps for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, X.select_dtypes(include=['number']).columns),
    ('cat', categorical_transformer, X.select_dtypes(include=['object']).columns)])

# Define models to evaluate
models = [
    LogisticRegression(random_state=42),
    KNeighborsClassifier(),
    LinearSVC(random_state=42)]

# Evaluate models using cross_val_score with accuracy and f1_score metrics
for model in models:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])
    acc_scores = cross_val_score(pipe, X, y, cv=5, scoring='accuracy')
    f1_scores = cross_val_score(pipe, X, y, cv=5, scoring='f1')
    print(f'{model.__class__.__name__} accuracy scores: {acc_scores}')
    print(f'{model.__class__.__name__} f1 scores: {f1_scores}')
    print(f'Mean accuracy: {acc_scores.mean():.3f}')
    print(f'Mean f1: {f1_scores.mean():.3f}\n')

D:\soft\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\soft\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optim

LogisticRegression accuracy scores: [0.84901218 0.84850036 0.85708436 0.85145373 0.8499181 ]
LogisticRegression f1 scores: [0.90367661 0.90321737 0.90900795 0.90504548 0.90417048]
Mean accuracy: 0.851
Mean f1: 0.905



MemoryError: Unable to allocate 512. MiB for an array with shape (134215755,) and data type int32

In [9]:
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,1
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,1
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,1
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,1
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,1


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC


# Replace '?' with most frequent values
data = data.replace('?', pd.Series(data.mode().values.flatten()))

# Split the data into features and target
X = data.drop(['class'], axis=1)
y = (data['class'] == '>50K').astype(int)

# Define the numerical and categorical transformers
num_transformer = make_pipeline(SimpleImputer(strategy='median'), MinMaxScaler())
cat_transformer = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder())

# Use ColumnTransformer to apply the transformers to the correct columns
preprocessor = make_column_transformer((num_transformer, X.select_dtypes(include='number').columns),
                                       (cat_transformer, X.select_dtypes(include='object').columns))

# Create pipelines for LogisticRegression, KNeighborsClassifier, and LinearSVC models
lr_pipeline = make_pipeline(preprocessor, LogisticRegression())
knn_pipeline = make_pipeline(preprocessor, KNeighborsClassifier())
svm_pipeline = make_pipeline(preprocessor, LinearSVC())

# Compute cross_val_score using accuracy and f1_score metrics
accuracy_lr = cross_val_score(lr_pipeline, X, y, cv=5, scoring='accuracy').mean()
f1_lr = cross_val_score(lr_pipeline, X, y, cv=5, scoring='f1').mean()

accuracy_knn = cross_val_score(knn_pipeline, X, y, cv=5, scoring='accuracy').mean()
f1_knn = cross_val_score(knn_pipeline, X, y, cv=5, scoring='f1').mean()

accuracy_svm = cross_val_score(svm_pipeline, X, y, cv=5, scoring='accuracy').mean()
f1_svm = cross_val_score(svm_pipeline, X, y, cv=5, scoring='f1').mean()

f1_LR = 0
acc_LR = 0
f1_KNN = 0
acc_KNN = 0
f1_SVM = 0
acc_SVM = 0
results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1_LR, acc_LR]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1_KNN, acc_KNN]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1_SVM, acc_SVM]

15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [164]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score


data = pd.read_csv('adult.csv')
data = data.replace('?', np.nan)
data = data.dropna()
data['class'] = data['class'].map({'<=50K': 1, '>50K': 0})
X = data.drop('class', axis=1)
y = data['class']


num_cols = X.select_dtypes(include='number').columns.tolist()
cat_cols = X.select_dtypes(include='object').columns.tolist()
print(num_cols)
print(cat_cols)


preprocessor = make_pipeline(
    OneHotEncoder(handle_unknown='ignore'),
    MinMaxScaler()
)

# Logistic Regression
lr_pipeline = make_pipeline(preprocessor, LogisticRegression())
lr_accuracy = cross_val_score(lr_pipeline, X, y, cv=5, scoring='accuracy').mean()
lr_f1 = cross_val_score(lr_pipeline, X, y, cv=5, scoring='f1').mean()

# K-Nearest Neighbors
knn_pipeline = make_pipeline(preprocessor, KNeighborsClassifier())
knn_accuracy = cross_val_score(knn_pipeline, X, y, cv=5, scoring='accuracy').mean()
knn_f1 = cross_val_score(knn_pipeline, X, y, cv=5, scoring='f1').mean()

# Linear SVC
svc_pipeline = make_pipeline(preprocessor, LinearSVC())
svc_accuracy = cross_val_score(svc_pipeline, X, y, cv=5, scoring='accuracy').mean()
svc_f1 = cross_val_score(svc_pipeline, X, y, cv=5, scoring='f1').mean()

print("Logistic Regression: Accuracy - {}, F1 score - {}".format(lr_accuracy, lr_f1))
print("K-Nearest Neighbors: Accuracy - {}, F1 score - {}".format(knn_accuracy, knn_f1))
print("Linear SVC: Accuracy - {}, F1 score - {}".format(svc_accuracy, svc_f1))


# # выводим результаты
# print('Logistic Regression Scores:', lr_scores)
# print('KNN Scores:', knn_scores)
# print('SVM Scores:', svm_scores)

# f1_LR_del_missings = 0
# acc_LR_del_missings = 0
# f1_KNN_del_missings = 0
# acc_KNN_del_missings = 0
# f1_SVM_del_missings = 0
# acc_SVM_del_missings = 0
# results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
# results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
# results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']


D:\soft\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\soft\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\soft\Anaconda\lib\site-packages\sklearn\pipeline.py", line 330, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "D:\soft\Anaconda\lib\site-packages\sklearn\pipeline.py", line 292, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "D:\soft\Anaconda\lib\site-packages\joblib\memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "D:\soft\Anaconda\lib\site-packages\sklearn\pipeline.py", line 740, in _fit_transform_one
    res = transformer.fit_transform(X, y, **fit_params)
  File "D:\soft\Anaconda\lib\site-packages\sklearn\pipe

Logistic Regression: Accuracy - nan, F1 score - nan
K-Nearest Neighbors: Accuracy - nan, F1 score - nan
Linear SVC: Accuracy - nan, F1 score - nan


D:\soft\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\soft\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\soft\Anaconda\lib\site-packages\sklearn\pipeline.py", line 330, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "D:\soft\Anaconda\lib\site-packages\sklearn\pipeline.py", line 292, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "D:\soft\Anaconda\lib\site-packages\joblib\memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "D:\soft\Anaconda\lib\site-packages\sklearn\pipeline.py", line 740, in _fit_transform_one
    res = transformer.fit_transform(X, y, **fit_params)
  File "D:\soft\Anaconda\lib\site-packages\sklearn\pipe

In [151]:
data = pd.read_csv('adult.csv')
data.mask(data.eq('?')).dropna()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48836,33,Private,245211,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,<=50K
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [ ]:


### Ваш код ###

f1_RF = 0
acc_RF = 0
f1_GB = 0
acc_GB = 0
results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [ ]:


### Ваш код ###

best_params = {}
print('Параметры лучшей модели:\n', best_params)
f1_best = 0
acc_best = 0
results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]

In [ ]:
results_classification